In [ ]:
import pathlib
import os

import cv2
import pandas as pd 
import numpy as np
import albumentations
import matplotlib.pyplot as plt

PROJECT_ROOT = pathlib.Path('./../..').resolve()

DATA_DIR = PROJECT_ROOT / 'SignDetectorAndClassifier' / 'data'
DATASET_DIR = DATA_DIR / 'YOLO_DATASET'

### Winter photo example for `classifier`

![](./../data/winter_traffic_signs_example/2_cut.png)
![](./../data/winter_traffic_signs_example/1_cut.png)

In [ ]:
img = cv2.imread(
    str(DATA_DIR / 'STOCK_SIGNS' / '1.8.png')
)
img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)

plt.imshow(img)

In [ ]:
class Winterize(albumentations.ImageOnlyTransform):
    def __init__(self, contrast_limit=0.8, tiles_count=5, tiles_size=5, tiles_relative=True, always_apply=False, p=0.5):
        super(Winterize, self).__init__(always_apply, p)
        self.contrast_limit = contrast_limit
        self.tiles_count = tiles_count
        self.tiles_size = tiles_size,
        self.tiles_relative = tiles_relative
        
    # TODO: implement white noise + gaussian on alpha channel of input image
    # TODO: implement while-gray random cuts from image
    def _transform(self, img):
        self.noise = np.random.normal(1, 1, img.shape)
        img = albumentations.brightness_contrast_adjust(img)
        return self.noise 

    def apply(self, img, clip_limit=2, **params):
        if albumentations.get_num_channels(img) != 4:
            raise TypeError("Winterize transformation expects RGBA image")

        return self._transform(img)

    def get_params(self):
        return {'contrast_limit': self.contrast_limit, 'tiles_count': self.tiles_count, 'tiles_size': self.tiles_size, 'tiles_relation': self.tiles_relative}

    def get_transform_init_args_names(self):
        return ("contrast_limit", "tiles_count", "tiles_size", "tiles_relative")

a = Winterize()

img_t = a.apply(img)
plt.imshow(img_t)

### Winter photo example for `detector`
![](./../data/winter_traffic_signs_example/1_full.png)

In [ ]:
import pathlib
import os

import cv2
import pandas as pd 
import numpy as np
import albumentations


PROJECT_ROOT = pathlib.Path('./../../').resolve()

DATA_DIR = PROJECT_ROOT / 'SignDetectorAndClassifier' / 'data'
DATASET_DIR = DATA_DIR / 'YOLO_DATASET'
DATASET_DIR

In [ ]:
USER_FULL_FRAMES = DATASET_DIR / 'USER_FULL_FRAMES' 
SAMPLE_IMG = USER_FULL_FRAMES / 'autosave01_02_2012_09_13_36.jpg'

from enum import Enum

class Season(Enum): 
    Winter = 1
    Fall = 2
    Summer = 3
    Autumn = 4
    
def extract_season_from_name(filepath: pathlib.Path) -> Season:
    name = filepath.stem 
    month = int(
        name.replace('autosave', '').replace('_', ' ').split(' ')[1]
    )
    if month in [12, 1, 2]:
        return Season.Winter
    if month in [3, 4, 5]:
        return Season.Autumn
    if month in [6, 7, 8]:
        return Season.Summer
    if month in [9, 10, 11]:
        return Season.Fall
    
    raise ValueError(f'Invalid month {month}')

files = USER_FULL_FRAMES.iterdir()
seasons = list(map(extract_season_from_name, files))
print(*[f'\t{x} count {seasons.count(x)}\n' for x in Season])



In [ ]:
import ast

def read_yolo_dataset(csv_path: pathlib.Path, filepath_prefix: str):
    data = pd.read_csv(csv_path)
    data['filepath'] = data['filepath'].apply(lambda x: pathlib.Path(filepath_prefix) / x)
    data['size'] = data['size'].apply(lambda x: ast.literal_eval(x))
    data['coords'] = data['coords'].apply(lambda x: ast.literal_eval(x))
    return data

yolo_dataset = read_yolo_dataset(DATASET_DIR / 'USER_FULL_FRAMES.csv', DATA_DIR)
yolo_dataset

## Per season split

In [ ]:
from collections import Counter
import seaborn as sns

hist_data = []
set_ = list(set(yolo_dataset['set']))
for idx, x in enumerate(set_):
    count_data = dict(
        Counter(map(extract_season_from_name, yolo_dataset[yolo_dataset['set'] == x]['filepath'].to_list())
        )
    )
    count_data = {str(x).split('.')[-1]: y for x, y in count_data.items()}
    hist_data.append(sorted([key for key, val in count_data.items() for _ in range(val)])) #, bins=set(count_data.keys()))

    # sns.countplot(data=df, x='Season')
plt.hist(hist_data, bins=len(set_), align='mid')
plt.legend(set_)
plt.show()

In [ ]:
hist_data

In [ ]:
import torch
# TODO: write dataset with winter augmentation
class WinterizeYoloDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        df,
        set_label,
        hyp_arg,
        img_size=640,
        batch_size=16,
        augment=False,
        hyp=None,
    ):

        self.img_size = img_size
        self.augment = augment
        self.hyp = hyp_arg

        self.df = df[df["set"] == set_label]
        self.albumentations = Albumentations() if augment else None

    def loadImage(self, instance):
        path, (w0, h0) = instance["filepath"], instance["size"]
        im = cv2.imread(path)
        assert im is not None, f"Image Not Found {path}"

        r = self.img_size / max(h0, w0)  # ratio

        if r != 1:  # if sizes are not equal
            im = cv2.resize(
                im,
                (int(w0 * r), int(h0 * r)),
                interpolation=cv2.INTER_AREA
                if r < 1 and not self.augment
                else cv2.INTER_LINEAR,
            )
        return im, (h0, w0), im.shape[:2]

    def __getitem__(self, index):

        # locate img info from DataFrame
        instance = self.df.iloc[index]

        # get Img, src height, width and resized height, width
        img, (h0, w0), (h, w) = self.loadImage(instance)

        shape = self.img_size

        # make img square
        # print('>', (img>1).sum())
        # print('<=', (img<=1).sum())
        img, ratio, pad = letterbox(img, shape, auto=False, scaleup=self.augment)
        # print(pad)
        # store core shape info
        shapes = (h0, w0), ((h / h0, w / w0), pad)  # for COCO mAP rescaling

        # add class to labels. We have 1 class, so just add zeros into first column
        labels = np.array(instance["coords"])
        labels = np.c_[np.zeros(labels.shape[0]), labels]
        # print(labels)

        # fix labels location caused by letterbox
        labels[:, 1:] = xywhn2xyxy(
            labels[:, 1:], ratio[0] * w, ratio[1] * h, padw=pad[0], padh=pad[1]
        )

        if self.augment:
            img, labels = random_perspective(
                img,
                labels,
                degrees=self.hyp["degrees"],
                translate=self.hyp["translate"],
                scale=self.hyp["scale"],
                shear=self.hyp["shear"],
                perspective=self.hyp["perspective"],
            )

        labels[:, 1:5] = xyxy2xywhn(
            labels[:, 1:5], w=img.shape[1], h=img.shape[0], clip=False, eps=1e-3
        )

        # YOLO augmentation technique (!copy-paste!)
        if self.augment:
            # print('augm for', index, instance['filepath'])
            # Albumentations
            img, labels = self.albumentations(img, labels)
            nl = len(labels)  # update after albumentations

            # HSV color-space
            augment_hsv(
                img,
                hgain=self.hyp["hsv_h"],
                sgain=self.hyp["hsv_s"],
                vgain=self.hyp["hsv_v"],
            )

            # Flip up-down
            if random.random() < self.hyp["flipud"]:
                img = np.flipud(img)
                if nl:
                    labels[:, 2] = 1 - labels[:, 2]

            # Flip left-right
            if random.random() < self.hyp["fliplr"]:
                img = np.fliplr(img)
                if nl:
                    labels[:, 1] = 1 - labels[:, 1]

        nl = len(labels)

        # why out size (?, 6)??
        labels_out = torch.zeros((nl, 6))
        if nl:
            labels_out[:, 1:] = torch.from_numpy(labels)

        img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
        img = np.ascontiguousarray(img)

        return torch.from_numpy(img), labels_out, instance["filepath"], shapes

    def __len__(self):
        return len(self.df.index)

    @staticmethod
    def collate_fn(batch):
        img, label, path, shapes = zip(*batch)  # transposed
        for i, l in enumerate(label):
            l[:, 0] = i  # add target image index for build_targets()
        return torch.stack(img, 0), torch.cat(label, 0), path, shapes


class WinterizeFull(albumentations.ImageOnlyTransform):
    """TODO: replace with decorator."""
    def __init__(
        self, 
        name_to_season_dict,
        transform: albumentations.ImageOnlyTransform,
    ):
        super(Winterize, self).__init__(True, 1)
    
    def _transform(self, img):
        self.noise = np.random.normal(1, 1, img.shape)
        img = albumentations.brightness_contrast_adjust(img)
        return self.noise 

    def apply(self, img, clip_limit=2, **params):
        if albumentations.get_num_channels(img) != 4:
            raise TypeError("Winterize transformation expects RGBA image")

        return self._transform(img)

    def get_params(self):
        return {'contrast_limit': self.contrast_limit, 'tiles_count': self.tiles_count, 'tiles_size': self.tiles_size, 'tiles_relation': self.tiles_relative}

    def get_transform_init_args_names(self):
        return ("contrast_limit", "tiles_count", "tiles_size", "tiles_relative")

a = Winterize()

img_t = a.apply(img)
plt.imshow(img_t)